# What Makes a Good Deal?
What makes an item offered for sale a good deal is a combination of **price** and **item type** (laptop, car, dress, table, ...).  Demand on some items may vary seasonally and geographically too, affecting the price accordingly.

In our training and test data we have the item's `price` and its geographical location (`city`, `region`).  We can identify its type either using the `category_name`feature, one of the `param_x` features, or using image recognition on the ad's `image`.

Let's see how these categorical features interact with `price` affecting `deal_probability` and pick one of them to cross with `price` as our new Good Deal feature.

First, we need to clean the `price` variable and remove outliers and missing values.

In [ ]:
"""Copy Keras pre-trained model files to work directory from:
https://www.kaggle.com/gaborfodor/keras-pretrained-models

Code from: https://www.kaggle.com/classtag/extract-avito-image-features-via-keras-vgg16/notebook
"""
import os

cache_dir = os.path.expanduser(os.path.join('~', '.keras'))
if not os.path.exists(cache_dir):
    os.makedirs(cache_dir)

# Create symbolic links for trained models.
# Thanks to Lem Lordje Ko for the idea
# https://www.kaggle.com/lemonkoala/pretrained-keras-models-symlinked-not-copied
models_symlink = os.path.join(cache_dir, 'models')
if not os.path.exists(models_symlink):
    os.symlink('/kaggle/input/keras-pretrained-models/', models_symlink)

images_dir = os.path.expanduser(os.path.join('~', 'avito_images'))
if not os.path.exists(images_dir):
    os.makedirs(images_dir)

In [ ]:
"""Extract images from Avito's advertisement image zip archive.

Code adapted from: https://www.kaggle.com/classtag/extract-avito-image-features-via-keras-vgg16/notebook
"""
import zipfile

NUM_IMAGES_TO_EXTRACT = 100

with zipfile.ZipFile('../input/avito-demand-prediction/train_jpg.zip', 'r') as train_zip:
    files_in_zip = sorted(train_zip.namelist())
    for idx, file in enumerate(files_in_zip[:NUM_IMAGES_TO_EXTRACT]):
        if file.endswith('.jpg'):
            train_zip.extract(file, path=file.split('/')[3])

!mv *.jpg/data/competition_files/train_jpg/* ~/avito_images
!rm -rf *.jpg

In [ ]:
def get_avito_image(image_id):
    """Read image file without extracting it from Avito's training images archive file and return it.
    
    Args:
        image_id (str): Id of the image to read.
        
    Returns:
       file: Image file.  Can be read with plt.imread() 
    """
    with zipfile.ZipFile('../input/avito-demand-prediction/train_jpg.zip', 'r') as train_zip:
        archive_member = os.path.join('data/competition_files/train_jpg', image_id + '.jpg')
        member = train_zip.open(archive_member)
    return member

def is_image_extracted(image_id):
    """Check if extracted copy of the image exists.
    
    Args:
        image_id (str): Image id
        
    Returns:
        bool: True if extracted copy of the image exists.
    """
    dir_path = os.path.expanduser('~/avito_images')
    return os.path.exists(os.path.join(dir_path, 'data/competition_files/train_jpg' , image_id + '.jpg'))


def extract_avito_images(image_ids):
    """Extract images from Avito training images and return extracted images path.
    
    If extracted copies of all images exist, returns their path instead.
    Args:
        image_ids (list): Image ids to extract.
    
    Returns:
        list: Extracted images path.
    """
    ex = []
    if all([is_image_extracted(x) for x in image_ids]):
        dir_path = os.path.expanduser('~/avito_images')
        for image_id in image_ids:
            ex.append(os.path.join(dir_path, 'data/competition_files/train_jpg' , image_id + '.jpg'))
        return ex
    
    with zipfile.ZipFile('../input/avito-demand-prediction/train_jpg.zip', 'r') as train_zip:
        for image_id in image_ids:
            archive_member = os.path.join('data/competition_files/train_jpg', image_id + '.jpg')
            dest = os.path.expanduser('~/avito_images')
            if os.path.exists(os.path.join(dest, archive_member)):
                e = os.path.join(dest, archive_member)
            else:        
                e = train_zip.extract(archive_member, dest)
            ex.append(e)
    return ex


def show_avito_images(image_ids):
    """Plot images from Avito's image data set.
    
    Args:
        image_id (list): Ids of the images to plot.
        
    Returns:
        matplotlib axis
    """
    ncols = 3
    nrows = len(image_ids) // ncols + 1 - (len(image_ids) % ncols == 0)
    images_path = extract_avito_images(image_ids)
    fig, axes = plt.subplots(nrows, ncols, figsize=(20, 20))
    for i, (image_id, image_path) in enumerate(zip(image_ids, images_path)):
        image = Image.open(image_path)
        ax = axes[i//ncols, i%ncols]
        ax.imshow(image)
        ax.set_title(image_id)
        ax.axis('off')
    plt.show()
    return ax

In [ ]:
# Dictionary of category names (Russian to English)
parent_cat_rus_eng = {
    '§­§Ú§é§ß§í§Ö §Ó§Ö§ë§Ú': 'Personal items',
    '§¥§Ý§ñ §Õ§à§Þ§Ñ §Ú §Õ§Ñ§é§Ú': 'For home and cottages',
    '§¢§í§ä§à§Ó§Ñ§ñ §ï§Ý§Ö§Ü§ä§â§à§ß§Ú§Ü§Ñ': 'Consumer electronics',
    '§¯§Ö§Õ§Ó§Ú§Ø§Ú§Þ§à§ã§ä§î': 'Real estate',
    '§·§à§Ò§Ò§Ú §Ú §à§ä§Õ§í§ç': 'Hobbies and recreation',
    '§´§â§Ñ§ß§ã§á§à§â§ä': 'Transportation',
    '§µ§ã§Ý§å§Ô§Ú': 'Services',
    '§¨§Ú§Ó§à§ä§ß§í§Ö':'Animals',
    '§¥§Ý§ñ §Ò§Ú§Ù§ß§Ö§ã§Ñ': 'For business'
}

# Dictionary of parent category names (Russian to English)
cat_rus_en = {
    '§¡§Ó§ä§à§Þ§à§Ò§Ú§Ý§Ú': 'Cars',
     '§¡§Ü§Ó§Ñ§â§Ú§å§Þ': 'Aquarium',
     '§¡§å§Õ§Ú§à §Ú §Ó§Ú§Õ§Ö§à': 'Audio and video',
     '§¢§Ú§Ý§Ö§ä§í §Ú §á§å§ä§Ö§ê§Ö§ã§ä§Ó§Ú§ñ': 'Tickets and travel',
     '§¢§í§ä§à§Ó§Ñ§ñ §ä§Ö§ç§ß§Ú§Ü§Ñ': 'Household appliances',
     '§£§Ö§Ý§à§ã§Ú§á§Ö§Õ§í': 'Bicycles',
     '§£§à§Õ§ß§í§Û §ä§â§Ñ§ß§ã§á§à§â§ä': 'Water transport',
     '§¤§Ñ§â§Ñ§Ø§Ú §Ú §Þ§Ñ§ê§Ú§ß§à§Þ§Ö§ã§ä§Ñ': 'Garages and Parking spaces',
     '§¤§à§ä§à§Ó§í§Û §Ò§Ú§Ù§ß§Ö§ã': 'Ready business',
     '§¤§â§å§Ù§à§Ó§Ú§Ü§Ú §Ú §ã§á§Ö§è§ä§Ö§ç§ß§Ú§Ü§Ñ': 'Trucks and machinery',
     '§¥§Ö§ä§ã§Ü§Ñ§ñ §à§Õ§Ö§Ø§Õ§Ñ §Ú §à§Ò§å§Ó§î': "Children's clothing and footwear",
     '§¥§à§Þ§Ñ, §Õ§Ñ§é§Ú, §Ü§à§ä§ä§Ö§Õ§Ø§Ú': 'Houses, cottages, cottages',
     '§¥§â§å§Ô§Ú§Ö §Ø§Ú§Ó§à§ä§ß§í§Ö': 'Other animals',
     '§©§Ö§Þ§Ö§Ý§î§ß§í§Ö §å§é§Ñ§ã§ä§Ü§Ú': 'Land plots',
     '§ª§Ô§â§í, §á§â§Ú§ã§ä§Ñ§Ó§Ü§Ú §Ú §á§â§à§Ô§â§Ñ§Þ§Þ§í': 'Games, consoles and programs',
     '§¬§Ó§Ñ§â§ä§Ú§â§í': 'Apartments',
     '§¬§ß§Ú§Ô§Ú §Ú §Ø§å§â§ß§Ñ§Ý§í': 'Books and magazines',
     '§¬§à§Ý§Ý§Ö§Ü§è§Ú§à§ß§Ú§â§à§Ó§Ñ§ß§Ú§Ö': 'Collecting',
     '§¬§à§Þ§Þ§Ö§â§é§Ö§ã§Ü§Ñ§ñ §ß§Ö§Õ§Ó§Ú§Ø§Ú§Þ§à§ã§ä§î': 'Commercial real estate',
     '§¬§à§Þ§ß§Ñ§ä§í': 'Rooms',
     '§¬§à§ê§Ü§Ú': 'Cats',
     '§¬§â§Ñ§ã§à§ä§Ñ §Ú §Ù§Õ§à§â§à§Ó§î§Ö': 'Health and beauty',
     '§®§Ö§Ò§Ö§Ý§î §Ú §Ú§ß§ä§Ö§â§î§Ö§â': 'Furniture and interior',
     '§®§à§ä§à§è§Ú§Ü§Ý§í §Ú §Þ§à§ä§à§ä§Ö§ç§ß§Ú§Ü§Ñ': 'Motorcycles and motor vehicles',
     '§®§å§Ù§í§Ü§Ñ§Ý§î§ß§í§Ö §Ú§ß§ã§ä§â§å§Þ§Ö§ß§ä§í': 'Musical instruments',
     '§¯§Ñ§ã§ä§à§Ý§î§ß§í§Ö §Ü§à§Þ§á§î§ð§ä§Ö§â§í': 'Desktop computers',
     '§¯§Ö§Õ§Ó§Ú§Ø§Ú§Þ§à§ã§ä§î §Ù§Ñ §â§å§Ò§Ö§Ø§à§Þ': 'Real estate abroad',
     '§¯§à§å§ä§Ò§å§Ü§Ú': 'Laptops',
     '§°§Ò§à§â§å§Õ§à§Ó§Ñ§ß§Ú§Ö §Õ§Ý§ñ §Ò§Ú§Ù§ß§Ö§ã§Ñ': 'Business equipment',
     '§°§Õ§Ö§Ø§Õ§Ñ, §à§Ò§å§Ó§î, §Ñ§Ü§ã§Ö§ã§ã§å§Ñ§â§í': 'Clothes, shoes, accessories',
     '§°§â§Ô§ä§Ö§ç§ß§Ú§Ü§Ñ §Ú §â§Ñ§ã§ç§à§Õ§ß§Ú§Ü§Ú': 'Office equipment and consumables',
     '§°§ç§à§ä§Ñ §Ú §â§í§Ò§Ñ§Ý§Ü§Ñ': 'Hunting and fishing',
     '§±§Ý§Ñ§ß§ê§Ö§ä§í §Ú §ï§Ý§Ö§Ü§ä§â§à§ß§ß§í§Ö §Ü§ß§Ú§Ô§Ú': 'Tablets and e-books',
     '§±§à§ã§å§Õ§Ñ §Ú §ä§à§Ó§Ñ§â§í §Õ§Ý§ñ §Ü§å§ç§ß§Ú': 'Kitchen utensils and goods',
     '§±§â§Ö§Õ§Ý§à§Ø§Ö§ß§Ú§Ö §å§ã§Ý§å§Ô': 'Offer of services',
     '§±§â§à§Õ§å§Ü§ä§í §á§Ú§ä§Ñ§ß§Ú§ñ': 'Food',
     '§±§ä§Ú§è§í': 'Birds',
     '§²§Ñ§ã§ä§Ö§ß§Ú§ñ': 'Plants',
     '§²§Ö§Þ§à§ß§ä §Ú §ã§ä§â§à§Ú§ä§Ö§Ý§î§ã§ä§Ó§à': 'Repair and construction',
     '§³§à§Ò§Ñ§Ü§Ú': 'Dogs',
     '§³§á§à§â§ä §Ú §à§ä§Õ§í§ç': 'Sport and recreation',
     '§´§Ö§Ý§Ö§æ§à§ß§í': 'Phones',
     '§´§à§Ó§Ñ§â§í §Õ§Ý§ñ §Õ§Ö§ä§Ö§Û §Ú §Ú§Ô§â§å§ê§Ü§Ú': 'Goods for children and toys',
     '§´§à§Ó§Ñ§â§í §Õ§Ý§ñ §Ø§Ú§Ó§à§ä§ß§í§ç': 'Animal products',
     '§´§à§Ó§Ñ§â§í §Õ§Ý§ñ §Ü§à§Þ§á§î§ð§ä§Ö§â§Ñ': 'Goods for computer',
     '§¶§à§ä§à§ä§Ö§ç§ß§Ú§Ü§Ñ': 'Photo equipment',
     '§¹§Ñ§ã§í §Ú §å§Ü§â§Ñ§ê§Ö§ß§Ú§ñ': 'Watches and jewelry'
}

In [ ]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
sns.set()
plt.rc('font', size=16)

%matplotlib inline

In [ ]:
usecols = ['item_id', 'parent_category_name', 'category_name', 'price', 'image']
train_df = pd.read_csv('../input/avito-demand-prediction/train.csv', usecols=usecols)
test_df = pd.read_csv('../input/avito-demand-prediction/test.csv', usecols=usecols)
df = pd.concat([train_df, test_df], axis=0)

df.category_name = df.category_name.map(lambda x: cat_rus_en[x])
df.parent_category_name = df.parent_category_name.map(lambda x: parent_cat_rus_eng[x])
df.head()

# Remove Extreme Prices
We have about 6% of the ads where the price is missing or is less than 10  &#8381; (Russian Rubles) (~ 0.16 USD).  These ads will be ignored from our analysis since they probably don't represent a reasonable asking price. 

Here is the distribution of the top 10 category names these cheap ads fall under, and their percentage make up.

In [ ]:
cheap = df[df.price < 10].category_name.value_counts().map(lambda x: '{:.2f}%'.format(x/df.shape[0] * 100))
print('Categories of items < 10 \u20BD (top 10)')
cheap.head(10)

Also, about 6% of the ads have price tage above 1 mllion &#8381; (~ 16,000 USD).  These are mosly real estate and cars with high price variance.  We will ignore these ads as well.

Here is the top 10 category distribution for these items and their percentage among all listed ads.

In [ ]:
expensive = df[df.price > 1000000].category_name.value_counts().map(lambda x: '{:.2f}%'.format(x/df.shape[0] * 100))
print('Categories of items > 1M \u20BD (top 10)')
expensive.head(10)

Overall, we drop 12% of the ads for this exercise.  This may seem a lot of wasted data, but I believe it's necessary to simplify this analysis and this feature creation.

In [ ]:
na_count = df.price.isna().sum()
zero_count = df[df.price < 10].price.count()
mill_count = df[df.price > 1000000].price.count()
total_count = df.shape[0]
print('Ads with missing price:\t{:,}\t({:.2f}%)'.format(na_count, na_count/total_count*100))
print('Ads with price < 10 \u20BD:\t{:,}\t({:.2f}%)'.format(zero_count, zero_count/total_count*100))
print('Ads with price > 1M \u20BD:\t{:,}\t({:.2f}%)'.format(mill_count, mill_count/total_count*100))
total_dropped = na_count + zero_count + mill_count
print('---------------------------------------')
print('Total dropped:\t\t{:,}\t({:.2f}%)'.format(total_dropped, total_dropped/total_count*100))

# Price Binning
A good item price binning should be based on the price distribution.  A histogram for the log value of the price for the remaining ads is shown below.  The bin boundaries are shown as vertical lines.

In [ ]:
dff = df[(df.price > 10) & (df.price < 1000000)] # Drop missing price and < 10 Ruble
price_log = np.log(dff.price)
out, bins = pd.cut(price_log, bins=100, retbins=True, labels=False)

plt.figure(figsize=(16, 5))
ax = sns.distplot(price_log, axlabel='Log(price)')
plt.title('Histogram of the Logarithmic value of the price.  Prices under 10 \u20BD and above 1M \u20BD are dropped.\nVertical lines show bin boundaries.')
plt.vlines(bins, color='g', ymin=0, ymax=0.1, alpha=0.3)
plt.show()

# Price Variance Within category_name
High price variance within an item category (`category_name`) may indicate the existance of implicit sub-categories causing the wide price range.  This behavior is undesirable for the Good Deal feature we are building here.  It is hard to have a good price estimate for an item if we only know the broad category it falls in.  For example, a `consumer_electronics` category will have a wide price range.  It's hard to estimate how good this deal is without knowing its subcategory (MacBookPro, speaker, TV, ...).

For a qualitative measure, the two violin plots below show how the log value of the price ( $Log(price)$ ) varies within each `category_name`.  The variation is large for about half of the categories.  Keep in mind that we are using a log scale for the price.

In [ ]:
cats = dff.category_name.unique()
dfa = dff[dff.category_name.isin(cats[0: 23])][['category_name', 'price']]
category_name = dfa.category_name
price_log = np.log(dfa.price)

plt.figure(figsize=(20, 8))
ax = sns.violinplot(x=category_name, y=price_log, scale='width', palette='Set3')
plt.ylim([1, 15])
plt.xticks(rotation=40, fontsize=14)
plt.ylabel('Log(price) \u20BD')
plt.title('Price variance within categories (1 of 2)')
plt.show()

In [ ]:
dfb = dff[dff.category_name.isin(cats[23: 47])][['category_name', 'price']]
category_name = dfb.category_name
price_log = np.log(dfb.price)
plt.figure(figsize=(20, 8))

ax = sns.violinplot(x=category_name, y=price_log, scale='width', palette='Set3')
plt.ylim([1, 15])
plt.xticks(rotation=40, fontsize=14)
plt.title('Price variance within categories (2 of 2)')
plt.ylabel('Log(price) \u20BD')
plt.show()

And then let's see how the price ( $Log(price)$ ) varies within the `parent_category_name` in the violin plot below.  Again, the variation looks relatively high for almost all the parent categories.

In [ ]:
parent_category = dff.parent_category_name
category = dff.category_name
price_log = np.log(dff.price)

plt.figure(figsize=(20, 8))
ax = sns.violinplot(x=parent_category, y=price_log, scale='width', palette='Set3')
plt.ylim([1, 15])
plt.xticks(rotation=40, fontsize=14)
plt.title('Price variance within parent categories')
plt.ylabel('Log(price) \u20BD')
plt.show()

For an objective measure, let's calculate the [coefficient of variation (CV)][cv] for our 47 categories.  A CV value greater than 1 suggests high relative variability in our variable.

As you can see blow, almost all categories have CV value greater than 1.

[cv]: http://www.statisticshowto.com/probability-and-statistics/how-to-find-a-coefficient-of-variation/

In [ ]:
print('Coefficient of variation (CV) for prices in different categories (category_name).')
dffd = dff.groupby('category_name')['price'].apply(lambda x: np.std(x)/np.mean(x)).sort_values(ascending=False)
dffd

----------------

# Use Ad Image to Identify Item Category
In [an earlier kernel][1] I compared three popular image recognition models on a small sample of images and found that [InceptionV3][inceptionv3] performed well.  This also aligns with [the models' accuracy as reported by Keras][2].  Therefore, we will be using InceptionV3 to recognize the item in the ad's image.

Due to computational resources limitations here on Kaggle, I ran this step locally to recognize the items in about 1.85 million images.  You can check out the recognized image items in this kernel's data set (`train_image_labels.csv`, `test_image_labels.csv`).

Let's see how well InceptionV3 did on this image recognition job.  Here are some images with their recognized labels.

[1]: https://www.kaggle.com/wesamelshamy/high-correlation-feature-image-classification-conf
[2]: https://keras.io/applications/#documentation-for-individual-models
[inceptionv3]: https://keras.io/applications/#inceptionv3

In [ ]:
train_image_labels = pd.read_csv('../input/avito-images-recognized/train_image_labels.csv', index_col='image_id')
test_image_labels = pd.read_csv('../input/avito-images-recognized/test_image_labels.csv', index_col='image_id')
all_image_labels = pd.concat([train_image_labels, test_image_labels], axis=0)

In [ ]:
from PIL import Image

dir_iter = os.scandir(os.path.expanduser('~/avito_images'))
fig, axes = plt.subplots(5, 5, figsize=(20, 20))
for i in range(25):
    e = next(dir_iter)
    img = Image.open(e.path)
    img = img.resize((360, 360))
    ax = axes[i//5, i%5]
    ax.imshow(img)
    im_id = e.name.split('.')[0]
    im_label = train_image_labels.loc[im_id]['image_label']
    ax.set_title(im_label, fontsize=24)
    ax.axis('off')
plt.show()

Overall, there were 996 identified items in the training and test datasets combined.  This is a good improvement over the 47 `category_name` classes we had earlier.

Let's see how the price varies within each one of these identified items.

# Price Variance Within Identified Items
Here are the recognized categories with the top 10 CV values.  We have 996 such categories.

In [ ]:
print('Coefficient of variation (CV) for prices in different recognized image categories.')
dfl = dff.merge(all_image_labels, left_on='image', right_index=True, how='left')
dfd = dfl.groupby('image_label')['price'].apply(lambda x: np.std(x)/np.mean(x)).sort_values(ascending=False)
dfd.head(10)

We now compare price variance in the recognized image categories and price variance in `category_name` categories.  The histogram below shows the distribution of the Coefficient of Variation (CV) for both cases.

A good distribution in our case should be right skewed (very thin and long right tail) indicating low price variance within categories.

In [ ]:
dfd2 = dfl.groupby('image_label')['price'].apply(lambda x: np.std(np.log(x))/np.mean(np.log(x)))
dffd2 = dfl.groupby('category_name')['price'].apply(lambda x: np.std(np.log(x))/np.mean(np.log(x)))

In [ ]:
plt.figure(figsize=(16, 5))
sns.set()
ax = sns.distplot(dfd2.values, color='darkred', hist_kws={'alpha': 0.3})
ax = sns.distplot(dffd2.values, color='darkgreen', hist_kws={'alpha': 0.3}, ax=ax)
ax.set_title('CV of image_lable and category_name')
ax.set_xlabel('CV of $Log(price)$')
ax.legend(['image label', 'category_name'])
plt.show()

# Price Variance in param_2
Finally, subcategory features `param_1`, `param_2` and `param_3` are other candidates for use with `price` to predict `deal_probability`.

One thing to note is that `param_2` and `param_3` are missing from about half the ads.  Also, `param_3` takes on values like {*small, 40-42 (XS), 39,..*} which could overlap with other categories.

Again, here is a histogram comparing CV distributions for all three cases (`image_label`, `category_name`, `param_2`).  The more right skewed the distribution is (longer and thinner right tail) the better.  That indicates lower CV values for more categories.

In [ ]:
train_all = pd.read_csv('../input/avito-demand-prediction/train.csv', usecols=['param_1', 'param_2', 'param_3', 'price', 'deal_probability'])
param2v = train_all[(train_all.price > 0) & (train_all.price < 1000000)].groupby('param_2')['price'].apply(lambda x: np.std(np.log(x))/np.mean(np.log(x)))

plt.figure(figsize=(16, 5))
sns.set()
ax = sns.distplot(dfd2.values, color='darkred', hist_kws={'alpha': 0.3})
ax = sns.distplot(dffd2.values, color='darkgreen', hist_kws={'alpha': 0.3}, ax=ax)
ax = sns.distplot(param2v.values, color='navy', hist_kws={'alpha': 0.3}, ax=ax)

ax.set_title('CV of image_label, category_name, and param_2')
ax.set_xlabel('CV of $Log(price)$')
ax.legend(['image label', 'category_name', 'param_2'])
plt.show()

`param_2` has the lowest CV values among the three, by a big margin.

# Effect of price on deal_probability
Now that we know that `param_2` has the best potential for identifying a good deal along with the price, let's see how their interaction affects `deal_probability`.

In the figure below we can clearly see the deal probability value drops as the price within the same `param_2` category goes up, making the deal less attractive.

In [ ]:
shoes = train_all[train_all.param_2 == '§°§Ò§å§Ó§î'][['price', 'deal_probability']]
outer = train_all[train_all.param_2 == '§£§Ö§â§ç§ß§ñ§ñ §à§Õ§Ö§Ø§Õ§Ñ'][['price', 'deal_probability']]
dress = train_all[train_all.param_2 == '§±§Ý§Ñ§ä§î§ñ §Ú §ð§Ò§Ü§Ú'][['price', 'deal_probability']]
knit = train_all[train_all.param_2 == '§´§â§Ú§Ü§à§ä§Ñ§Ø'][['price', 'deal_probability']]

plt.figure(figsize=(16, 5))
fontsize=18

ax1 = plt.subplot(121)
plt.scatter(shoes.price, shoes.deal_probability, s=3)
x = plt.setp(ax1.get_yticklabels(), fontsize=fontsize)
plt.xlim([0, 6000])
plt.xlabel('Price \u20BD', fontsize=fontsize)
plt.xticks(fontsize=fontsize)
plt.yticks(fontsize=fontsize)
plt.ylabel('Deal probability', fontsize=fontsize)
ax = plt.title('Shoes', fontsize=fontsize)

ax2 = plt.subplot(122)
plt.scatter(outer.price, outer.deal_probability, s=3)
plt.setp(ax2.get_yticklabels(), visible=False)
plt.xlim([0, 7000])
plt.xlabel('Price \u20BD', fontsize=fontsize)
plt.xticks(fontsize=fontsize)
ax = plt.title('Outerwear', fontsize=fontsize)

# ax3 = plt.subplot(133)
# plt.scatter(dress.price, dress.deal_probability, s=3)
# x = plt.setp(ax3.get_yticklabels(), visible=False)
# plt.xlim([0, 6000])
# plt.xlabel('Price \u20BD')
# ax = plt.title('Dresses')

# ax4 = plt.subplot(144, sharex=ax1, sharey=ax1)
# plt.scatter(knit.price, knit.deal_probability)
# x = plt.setp(ax4.get_yticklabels(), visible=False)
# plt.xlim([0, 2000])
# plt.xlabel('Price \u20BD')
# ax = plt.title('Knitwear')

plt.tight_layout()

# What Does This All Mean?
Having `param_2` and `price` as separate features in decision tree models help the model as we can see in [kxx's xbg model][kxx], [Lathwal's model][lath], and [Himanshu's LGM model][him] among many others.  They usually rank in the top 10 predictive features.

**Having a feature that is a corss of `param_2` and bucketized `price` should help more in predicting `deal_probability`.**

That was my analysis.  I don't think sharing a complete wokring model with code and everything helps us learn.  Now you can implement this feature yourself and share with us what you find.

Cheers

[kxx]: https://www.kaggle.com/kailex/xgb-text2vec-tfidf-0-2237
[lath]: https://www.kaggle.com/codename007/avito-eda-fe-time-series-dt-visualization
[him]: https://www.kaggle.com/him4318/avito-lightgbm-with-ridge-feature-v-2-0